# A simple tool calling agent

In the first stage of the building agents, we will create a tool calling agent. I will explain step by step, how you can make a simple but powerful AI application but letting it interact with the real world, empowering it with tools. And yes, everything without using any agentic framework. 

All you need, is an openAI LLM access :) 

Thanks and many credit goes to: https://github.com/neural-maze/agentic-patterns-course/tree/main

Thanks to Neural Maze YoutTube channel: https://www.youtube.com/@TheNeuralMaze

In [1]:
import json
from typing import Callable, List
import requests
from ddgs import DDGS
import re
import subprocess
from openai import OpenAI 
import os
import warnings
warnings.filterwarnings("ignore")

## **Step 1**: The following function will create a signature of the python function. Since,we will give our agents many tools to use, it is better if the tools have a schema. 

In [2]:
def get_fn_signature(fn: Callable) -> dict:
    """
    Generates the signature for a given function.

    Args:
        fn (Callable): The function whose signature needs to be extracted.

    Returns:
        dict: A dictionary containing the function's name, description,
              and parameter types.
    """
    fn_signature: dict = {
        "name": fn.__name__,
        "description": fn.__doc__,
        "parameters": {"properties": {}},
    }
    schema = {
        k: {"type": v.__name__} for k, v in fn.__annotations__.items() if k != "return"
    }
    fn_signature["parameters"]["properties"] = schema
    return fn_signature

## **Step 2**: We will build some tools

In [3]:
def count_letter_occurrence(text:str, letter:str)-> dict():
    """
    The function helps counting the number of occurrence of a letter in a word

    #Inputs:
    text: str, the word or sentence or the text whose specific letter is to be counted
    letter: str, the letter whose occurrence is to be counted

    # Output:
    {"occurrence_count":occurrence_count}
    How many times the letter occurs
    
    """
    occurrence_count = 0
    for L in text:
        if L==letter:
            occurrence_count += 1

    return {"occurrence_count":occurrence_count}



def find_location_of_a_file(filename:str)->str:
    """
    Search for the specified file on macOS using Spotlight's `mdfind` command.

    Finds the location of a file on macOS. The search is case-insensitive and
    will return all matching file paths if found.

    Parameters
    ----------
    filename : str
        The name of the file to search for (e.g., "document.pdf").
        
    Returns
    -------
    str
        - A newline-separated string containing one or more full file paths
          to matching files if found.
        - The string "Not found" if no matches are returned by `mdfind`.

    """
    out = subprocess.run(
        ["/usr/bin/mdfind", "-name", filename],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True,
        check=False)

    if out.stdout=="":
        return "Not found"
    else:
        return out.stdout
    

def news(what_news:str):
    """
    Upon having a news item to search, it returns a set of items about that news item
    """

    return DDGS().news(query=what_news)
    

## LLM Essentials

Get the key yourself :) 

In [4]:
## Set the API key and model name
MODEL="gpt-4o-mini"

os.environ["OPENAI_API_KEY"] = "sk-proj-xxxxxxxxxx"

client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

## **Step 3:** Build the system prompt to make an LLM a tool mapper.

Cool, isn't it?


In [5]:
TOOL_SYSTEM_PROMPT = f"""
You are a function calling AI model. You are provided with function signatures within <tools></tools> XML tags. 
You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug 
into functions. Pay special attention to the properties 'types'. You should use those types as in a Python dict.
For each function call return a json object with function name and arguments within <tool_call></tool_call> XML tags as follows:

<tool_call>
{{"name": <function-name>,"arguments": <args-dict>}}
</tool_call>

Here are the available tools:

{get_fn_signature(count_letter_occurrence)}

{get_fn_signature(find_location_of_a_file)}

{get_fn_signature(news)}


</tools>
"""

## **Step 4:** Once we have the tools, we will also build a tool mapper. 

You see, when the LLm will decide which tool to use, we also need a mapper. because the LLM will output string, but the tools are in python function. Hence a mapper is necessary

In [6]:
def tool_mapper(funcname, args):
    if funcname=="count_letter_occurrence":
        return count_letter_occurrence(**args)
    elif funcname=="news":
        return news(**args)
    elif funcname=="find_location_of_a_file":
        return find_location_of_a_file(**args)
    else:
        return "Not found"

## **Step 5:** You need to parse the output from the LLM. 


In [7]:
def parse_tool_call_str(tool_call_str: str):
    pattern = r'</?tool_call>'
    clean_tags = re.sub(pattern, '', tool_call_str)
    
    try:
        tool_call_json = json.loads(clean_tags)
        return tool_call_json, True
    except json.JSONDecodeError:
        return clean_tags, False
    except Exception as e:
        print(f"Unexpected error: {e}")
        return "There was some error parsing the Tool's output", False

## Step 6: Build the agent

We are now ready to build our tool calling agent. We only have 3 simple tools, but feel free to add as many as you want.
The agent will simply decide which tool to use. Then, once it calls  tool, we will have a system in place to execute it. Finally, we will clean the tool calling output and again call the LLM to ouput a human-readable clean version

In [8]:
def tool_calling_agent(Query:str="Who is winner in todays game", verbose=True):
    completion = client.chat.completions.create(
          model=MODEL,
          messages=[
            {"role": "system", "content": TOOL_SYSTEM_PROMPT}, # <-- This is the system message that provides context to the model
            {"role": "user", "content": Query}  # <-- This is the user message for which the model will generate a response
          ]
        
        )
    if verbose:
        print(f"===== First Completion Ouptut:\n{completion.choices[0].message.content}=====\n\n")
    parsed, status = parse_tool_call_str(completion.choices[0].message.content)
    if verbose:
        print(f"===== Here's the parsed Ouptut:\n{parsed}=====")
        
    if status==False:
        print(f"~~~~ The tool being used: None")
        return parsed
    output = tool_mapper(funcname=parsed['name'], args=parsed["arguments"])

    if verbose:
        print(f"==== Too output: {output}")
    
    full_response_prompt = f"""Question: {Query}\n\n
    Response: {output} \n\n
    summary or brief response:
    
    """
    
    completion2 = client.chat.completions.create(
          model=MODEL,
          messages=[
            {"role": "system", "content": "Given a question and the response, your job is to summarize or present a very brielf to the point answer"}, # <-- This is the system message that provides context to the model
            {"role": "user", "content": full_response_prompt}  # <-- This is the user message for which the model will generate a response
          ]
        
        )
    print(f"~~~~ The tool being used: {parsed['name']} ~~~~~\n\n\n")
    return completion2.choices[0].message.content

In [12]:
output = tool_calling_agent("In the game of Bangladesh vs Netherland T20i, who was the winner?", verbose=False)
print(output)

~~~~ The tool being used: news ~~~~~



Bangladesh won the T20I series against the Netherlands 2-0 after the final match was abandoned due to rain.


In [13]:
output = tool_calling_agent("How many r is there in Strawberry", verbose=False)
print(output)

~~~~ The tool being used: count_letter_occurrence ~~~~~



There are 3 'r's in Strawberry.


In [14]:
output = tool_calling_agent("I cannot find the file demo_v2.ipynb in my local computer :(", verbose=False)
print(output)

~~~~ The tool being used: find_location_of_a_file ~~~~~



The file demo_v2.ipynb can be found in the following locations:  
1. /Users/sadatsh/Downloads/demo_v2.ipynb  
2. /Users/sadatsh/Documents/GenesisApeScience/notebooks/demo_v2.ipynb  


In [15]:
output = tool_calling_agent("What is the capital of Quatar?", verbose=False)
print(output)

~~~~ The tool being used: None
The capital of Qatar is Doha.


In [16]:
output = tool_calling_agent("What is the latest news of Mexico?", verbose=True)
print(output)

===== First Completion Ouptut:
<tool_call>
{"name": "news","arguments": {"what_news": "Mexico"}}
</tool_call>=====


===== Here's the parsed Ouptut:
{'name': 'news', 'arguments': {'what_news': 'Mexico'}}=====
==== Too output: [{'date': '2025-09-04T01:17:00+00:00', 'title': '🥳 Party time: Mexico gears up for the start of World Cup 2026', 'body': 'The Head of Government, Clara Brugada, announced that June 11, 2026, the date on which the World Cup inauguration will take place at the Azteca Stadium, will be declared an official holiday in Mexico', 'url': 'https://sports.yahoo.com/article/party-time-mexico-gears-start-011200577.html', 'image': 'https://s.yimg.com/ny/api/res/1.2/PTwnJP.ZRVJlakPl5LZjyw--/YXBwaWQ9aGlnaGxhbmRlcjt3PTEyMDA7aD03Nzg7Y2Y9d2VicA--/https://media.zenfs.com/en/onefootball_articles_802/a3c9f2abb407e19ebabbd0bda18a56ec', 'source': 'Yahoo Sports'}, {'date': '2025-09-03T19:57:00+00:00', 'title': 'Rubio Reassures Mexico After U.S. Military Strike Jolts Region', 'body': 'Secr